## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
from nltk.corpus import stopwords
import re

## Data

In [2]:
df_a = pd.read_csv(r'C:\Users\stan\mimic-iii-data\ADMISSIONS.csv.gz')  # length of stay and id's
df_e = pd.read_csv(r'C:\Users\stan\mimic-iii-data\NOTEEVENTS.csv.gz')  # text data
df_p = pd.read_csv(r'C:\Users\stan\mimic-iii-data\PROCEDURES_ICD.csv.gz')  # procedure code for each patients
df_dp = pd.read_csv(r'C:\Users\stan\mimic-iii-data\D_ICD_PROCEDURES.csv.gz')  # procedure name
df_d = pd.read_csv(r'C:\Users\stan\mimic-iii-data\DIAGNOSES_ICD.csv.gz')  # diagnosis code for each patient
df_dd = pd.read_csv(r'C:\Users\stan\mimic-iii-data\D_ICD_DIAGNOSES.csv.gz')  # diagnosis name
df_pa = pd.read_csv(r'C:\Users\stan\mimic-iii-data\PATIENTS.csv.gz') # patients' information
df_is = pd.read_csv(r'C:\Users\stan\mimic-iii-data\ICUSTAYS.csv.gz')  # icu stay length

c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Taking one patient group

In [3]:
'''Only considering the patients who were discharged to Home Healthcare'''

df_hhc = df_a.loc[df_a.DISCHARGE_LOCATION == "HOME HEALTH CARE"]
df = pd.DataFrame()
df = df_hhc[["SUBJECT_ID", "HADM_ID","MARITAL_STATUS", "ADMITTIME", "DISCHTIME"]]
df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,SUBJECT_ID,HADM_ID,MARITAL_STATUS,ADMITTIME,DISCHTIME
0,23,152223,MARRIED,2153-09-03 07:15:00,2153-09-08 19:10:00
1,23,124321,MARRIED,2157-10-18 19:34:00,2157-10-25 14:00:00
2,28,162569,MARRIED,2177-09-01 07:15:00,2177-09-06 16:00:00
3,30,104557,MARRIED,2172-10-14 14:17:00,2172-10-19 14:37:00
4,34,144319,MARRIED,2191-02-23 05:23:00,2191-02-25 20:20:00


### Marital Status

In [4]:
'''Separating the marital status into categorical columns'''

for column in df["MARITAL_STATUS"].unique():
    df[column] = 0

for key in df["MARITAL_STATUS"].unique():
    for (i, value) in enumerate(df["MARITAL_STATUS"].values):
        if str(value) == key:
            df[key][i] = 1
df.drop("MARITAL_STATUS", axis = 1, inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
0,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,1,0,0,0,0,0,0
1,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,1,0,0,0,0,0,0
2,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,1,0,0,0,0,0,0
3,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,1,0,0,0,0,0,0
4,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,1,0,0,0,0,0,0


### IDs, Text input and Output (Length of stay)

In [5]:
df["LENGTH_OF_STAY"] = (pd.to_datetime(df["DISCHTIME"]) - pd.to_datetime(df["ADMITTIME"])).dt.days
df = df.merge(df_e[["HADM_ID","TEXT"]], left_on = "HADM_ID", right_on = "HADM_ID")
df.drop(['DISCHTIME', 'ADMITTIME'], 1, inplace = True)
df.head()

c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,SUBJECT_ID,HADM_ID,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER,LENGTH_OF_STAY,TEXT
0,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...
1,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nMarked left axis deviation\nRBBB...
2,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nSupraventricular extrasystoles\n...
3,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nMarked left axis deviation\nRBBB...
4,23,152223,1,0,0,0,0,0,0,5,Sinus arrhythmia\nLeft axis deviation\nIV cond...


In [6]:
df.TEXT[0]

'Admission Date:  [**2153-9-3**]              Discharge Date:   [**2153-9-8**]\n\nDate of Birth:  [**2082-7-17**]             Sex:   M\n\nService: CARDIOTHORACIC\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 1505**]\nChief Complaint:\nChest pain, SOB, positive ETT\n\nMajor Surgical or Invasive Procedure:\n[**2153-9-3**] Four vessel coronary artery bypass grafting(LIMA to LAD,\nSVG to Diagonal, SVG to OM, SVG to PDA)\n\nHistory of Present Illness:\nThis is a 71 year old male with known CAD. He underwent PTCA to\nLAD and diagonal in [**2145**]. Prior to hernia repair operation, an\nETT in [**2153-7-27**] was notable for EKG changes. An ECHO in [**Month (only) 205**]\n[**2153**] was notable for mild MR [**First Name (Titles) **] [**Last Name (Titles) 1192**] AS. The [**Location (un) 109**] was\nestimated at 1.1 cm2 with peak/mean gradients of 34 and 22 mmHg.\nThe was mild concentric LVH with an LVEF of 60%. He was\nsubsequently refe

### Length of ICU stay (Input)

In [7]:
df2 = df
df2 = df2.merge(df_is["LOS"], left_on = "HADM_ID", right_on = df_is["HADM_ID"])
df2.rename(columns={"LOS": "ICU_LOS"}, inplace = True)
df2.head()

,SUBJECT_ID,HADM_ID,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER,LENGTH_OF_STAY,TEXT,ICU_LOS
0,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641
1,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nMarked left axis deviation\nRBBB...,1.2641
2,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nSupraventricular extrasystoles\n...,1.2641
3,23,152223,1,0,0,0,0,0,0,5,Sinus rhythm\nMarked left axis deviation\nRBBB...,1.2641
4,23,152223,1,0,0,0,0,0,0,5,Sinus arrhythmia\nLeft axis deviation\nIV cond...,1.2641


### Diagnoses

In [8]:
df3 = df2
df3 = df3.merge(df_d[["ICD9_CODE"]], left_on = "HADM_ID", right_on = df_d["HADM_ID"], suffixes = ("_PROCEDURES", "_DIAGNOSES"))
df3.head()

,SUBJECT_ID,HADM_ID,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER,LENGTH_OF_STAY,TEXT,ICU_LOS,ICD9_CODE
0,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641,41401
1,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641,4111
2,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641,4241
3,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641,V4582
4,23,152223,1,0,0,0,0,0,0,5,Admission Date: [**2153-9-3**] D...,1.2641,2724


In [9]:
df4 = df3
df4 = df4.groupby('HADM_ID').agg({'SUBJECT_ID': 'first',
                                  'ICD9_CODE': 'count',
                                  'LENGTH_OF_STAY': 'first',
                                  'ICU_LOS': 'first',
                                  'TEXT': 'first',
                                  'MARRIED': 'first', 
                                  'DIVORCED': 'first', 
                                  'SINGLE': 'first',
                                  'SEPARATED': 'first', 
                                  'WIDOWED': 'first', 
                                  'UNKNOWN (DEFAULT)': 'first', 
                                  'LIFE PARTNER': 'first'})
df4.rename(columns={"ICD9_CODE": "NUMBER_OF_DIAGNOSIS"}, inplace= True)
df4.head()

,SUBJECT_ID,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
HADM_ID,,,,,,,,,,,,
100009,533,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0
100012,60039,165,10,3.5763,Admission Date: [**2177-3-12**] ...,1,0,0,0,0,0,0
100020,9973,738,10,1.1011,Admission Date: [**2142-11-30**] ...,1,0,0,0,0,0,0
100028,53456,99,6,2.0321,Admission Date: [**2142-12-23**] ...,0,0,1,0,0,0,0
100034,51495,88,3,1.1599,Admission Date: [**2164-4-23**] ...,1,0,0,0,0,0,0


### Procedures

In [26]:
df5 = df4
df5 = df5.merge(df_p["ICD9_CODE"], left_on = "HADM_ID", right_on = df_p["HADM_ID"])
df5.head()

,HADM_ID,SUBJECT_ID,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER,ICD9_CODE
0,100009,533,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0,3613
1,100009,533,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0,3615
2,100009,533,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0,3795
3,100009,533,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0,3961
4,100012,60039,165,10,3.5763,Admission Date: [**2177-3-12**] ...,1,0,0,0,0,0,0,3615


In [27]:
df5 = df5.groupby('HADM_ID').agg({'SUBJECT_ID': 'first',
                                  'ICD9_CODE': 'count',
                                  'NUMBER_OF_DIAGNOSIS': 'first',
                                  'LENGTH_OF_STAY': 'first',
                                  'ICU_LOS': 'first',
                                  'TEXT': 'first',
                                  'MARRIED': 'first', 
                                  'DIVORCED': 'first', 
                                  'SINGLE': 'first',
                                  'SEPARATED': 'first', 
                                  'WIDOWED': 'first', 
                                  'UNKNOWN (DEFAULT)': 'first', 
                                  'LIFE PARTNER': 'first'})
df5.rename(columns={"ICD9_CODE": "NUMBER_OF_PROCEDURES"}, inplace= True)
df5.dropna(inplace = True)
df5.head()

,SUBJECT_ID,NUMBER_OF_PROCEDURES,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
HADM_ID,,,,,,,,,,,,,
100009,533,4,144,4,2.4908,Admission Date: [**2162-5-16**] ...,1,0,0,0,0,0,0
100012,60039,4,165,10,3.5763,Admission Date: [**2177-3-12**] ...,1,0,0,0,0,0,0
100028,53456,5,99,6,2.0321,Admission Date: [**2142-12-23**] ...,0,0,1,0,0,0,0
100034,51495,3,88,3,1.1599,Admission Date: [**2164-4-23**] ...,1,0,0,0,0,0,0
100037,58947,13,6846,46,1.8369,Admission Date: [**2183-3-23**] ...,1,0,0,0,0,0,0


### Function: text processing

In [28]:
replace_space = re.compile('[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]')
clear_symbols = re.compile('[^0-9a-z#+_]')
remove_stopwords = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = replace_space.sub(' ', text)
    text = clear_symbols.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in remove_stopwords)
    return text

df5.TEXT = df5.TEXT.apply(clean_text)
df5.reset_index(inplace = True)
df5.head()

,HADM_ID,SUBJECT_ID,NUMBER_OF_PROCEDURES,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
0,100009,533,4,144,4,2.4908,admission date 2162 5 16 discharge date 2162 5...,1,0,0,0,0,0,0
1,100012,60039,4,165,10,3.5763,admission date 2177 3 12 discharge date 2177 3...,1,0,0,0,0,0,0
2,100028,53456,5,99,6,2.0321,admission date 2142 12 23 discharge date 2142 ...,0,0,1,0,0,0,0
3,100034,51495,3,88,3,1.1599,admission date 2164 4 23 discharge date 2164 4...,1,0,0,0,0,0,0
4,100037,58947,13,6846,46,1.8369,admission date 2183 3 23 discharge date 2183 5...,1,0,0,0,0,0,0


In [29]:
df5.TEXT[0]

'admission date 2162 5 16 discharge date 2162 5 21 date birth 2101 7 30 sex service cardiothoracic allergies known allergies adverse drug reactions attending first name3 lf 1505 chief complaint angina major surgical invasive procedure 2162 5 17 cabgx4 lima lad rsvg diagonal posterior descending artery obtuse marginal 2162 5 19 right atrial lead placement history present illness 60yo man known coronary disease ami 2143 name prefix prefixes last name prefixes lcx 2155 well last week developed angina initially exertion progressed rest angina episode releived sl ntg episode lasting 5 minutes presented cardiologist treatment admitted mwmc cardiac catheterization revealed 3 vessel disease transferred hospital1 18 coronary bypass grafting cardiac catheterization date 2162 5 11 place mwmc lad chronic total occlusion proximally distal filling via collaterals rca chronic total occlusion non dominant rca 90 lcx new complex 90 stenosis prox lcx involving bifurcation lcx proper large om2 old stent 

### Dropping texts shorter than the max sequence

In [14]:
df6 = df5
max_sequence = 300
for (pos, text) in enumerate(df6["TEXT"].values):
    text_length = len(text.split())
    if text_length <= (max_sequence):
        df6.drop([pos], axis = 0, inplace = True)
df6.head()

,HADM_ID,SUBJECT_ID,NUMBER_OF_PROCEDURES,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
0,100009,533,4,144,4,2.4908,admission date 2162 5 16 discharge date 2162 5...,1,0,0,0,0,0,0
1,100012,60039,4,165,10,3.5763,admission date 2177 3 12 discharge date 2177 3...,1,0,0,0,0,0,0
2,100028,53456,5,99,6,2.0321,admission date 2142 12 23 discharge date 2142 ...,0,0,1,0,0,0,0
3,100034,51495,3,88,3,1.1599,admission date 2164 4 23 discharge date 2164 4...,1,0,0,0,0,0,0
4,100037,58947,13,6846,46,1.8369,admission date 2183 3 23 discharge date 2183 5...,1,0,0,0,0,0,0


### Get sample from population Text

In [15]:
texts_data = []
for (i, text) in enumerate(df6["TEXT"].values):
    text_length = None
    chosen_text_splitted = None
    chosen_text_whole = None
    text_length = len(text.split())
    random_key = np.random.randint(0, (text_length - max_sequence))
    chosen_text_splitted = text.split()[random_key: (random_key + max_sequence)]
    chosen_text_whole = ' '.join(chosen_text_splitted)
    df6["TEXT"][i] = chosen_text_whole
df6.head()

c:\users\stan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,HADM_ID,SUBJECT_ID,NUMBER_OF_PROCEDURES,NUMBER_OF_DIAGNOSIS,LENGTH_OF_STAY,ICU_LOS,TEXT,MARRIED,DIVORCED,SINGLE,SEPARATED,WIDOWED,UNKNOWN (DEFAULT),LIFE PARTNER
0,100009,533,4,144,4,2.4908,grafting cardiac catheterization date 2162 5 1...,1,0,0,0,0,0,0
1,100012,60039,4,165,10,3.5763,6 mchc 36 6 rdw 13 5 2177 3 12 04 05pm hba1c 5...,1,0,0,0,0,0,0
2,100028,53456,5,99,6,2.0321,bolus lr bp improved 90 100 given ivf needed m...,0,0,1,0,0,0,0
3,100034,51495,3,88,3,1.1599,1 diuresed towards preoperative weight pod 1 t...,1,0,0,0,0,0,0
4,100037,58947,13,6846,46,1.8369,73 p 83 r 20 o2 975 ra general alert oriented ...,1,0,0,0,0,0,0


In [16]:
df6.to_csv('data_extracted.csv')